In [1]:
# import libraries
from binance import Client
import csv
import pandas as pd
from datetime import datetime as dt
import pandas_ta as ta
import numpy as np

In [2]:
client = Client(None, None)

In [3]:
# defining the function that fetch the historical data of the wanted coin
def getdata(symbol, period, start, end):
    """
    This function gives us the historical candle data of the desired coin
    :param symbol: coin name that you want to get data
    :param period: time period of candles
    :param start: starting date of candles data
    :param end: end date of candles data
    :return: candles data
    """
    candles = client.get_historical_klines(symbol, period, start, end)
    return candles

In [4]:
# defining the function that make csv file of the historical data of the wanted coin
def make_csv(symbol, candles):
    """
    This function makes csv file of desired coin with defined properties
    :param symbol: coin name that you want to make csv file
    :param candles: historical data of the desired coin
    :return: historical data in csv file
    """
    csvfile = open(symbol + ".csv", "a", newline="")
    cursor = csv.writer(csvfile)
    for i in candles:
        cursor.writerow(i)
    csvfile.close()

In [5]:
# defining the function that make csv files of the historical data of the wanted multiple coin
def multiple_csv(symbols, interval, start, end):
    """
    This function makes csv file for each coin in symbols parameter with defined properties
    :param symbols: list of multiple coin names that you want to make csv file
    :param interval: time period of candles (default: client.KLINE_INTERVAL_1DAY you can change the interval)
    :param start: starting date of candles data
    :param end: end date of candles data
    :return: historical data of multiple coin in csv files
    """
    for i in symbols:
        make_csv(i, getdata(i, interval, str(start), str(end)))
        print(i, "csv file is ready.")

In [6]:
# defining the function that make a coin list from binance include just USDT in their symbols
def make_coin_list():
    """
    This function makes a list that contain coin names just include USDT in their symbols
    :return: coin symbols list with just USDT
    """
    symbols = []
    info = client.get_all_tickers()
    for i in info:
        if "USDT" in i["symbol"]:
            if "DOWN" in i["symbol"] or "UP" in i["symbol"]:
                pass
            else:
                symbollist.append(i["symbol"])
    return symbols

In [7]:
# defining function the that turn the timestamp to the date 
def calculate_time(timestamp):
    """
    This function turns the timestamp to the date
    :param timestamp: given timestamp
    :return: date according to given timestamp
    """
    return dt.fromtimestamp(timestamp/1000)

In [8]:
# defining the function that make csv file for each coin that include USDT in their symbols
def all_usdt_data():
    """
    This function makes csv files for coins that include USDT in their symbols
    :return: csv files of historical data of all coins that include USDT in their symbols in Binance
    """
    for i in make_coin_list():
        multiple_csv([i], client.KLINE_INTERVAL_4HOUR, "8 November 2021", "8 September 2022")

In [9]:
multiple_csv(["BTCUSDT", "XRPUSDT", "DOTUSDT", "ETHUSDT"], client.KLINE_INTERVAL_4HOUR, "8 November 2021", "8 September 2022")

BTCUSDT csv file is ready.
XRPUSDT csv file is ready.
DOTUSDT csv file is ready.
ETHUSDT csv file is ready.


In [10]:
# example strategy function
def strategyA(coin:str):
    usdtpertrade = 100
    buycount = 0
    sellcount = 0
    totalcoin = 0
    comission_rate = 75/10000
    comission = 0
    coin_csv = coin + ".csv"
    headers = ["Open Time", "Open", "High", "Low", "Close", "Volume", "Close Time", "QAV", "NAT", "TBBAV", "TBQAV", "Ignore"]
    coin_data = pd.read_csv(coin_csv, names=headers)
    close = coin_data["Close"]
    o_time = coin_data["Open Time"]
    sma50 = ta.ma("sma", close, length=50)
    sma200 = ta.ma("sma", close, length=200)
    transactions = ["sell"]
    print("*********************************************************************")
    for i in range(len(close)):
        
        if pd.isna(sma50[i]) is False:
            
            if sma50[i-1] < sma200[i-1] and sma50[i] > sma200[i] and transactions[-1] != "buy":
                print("On", calculate_time(o_time[i+1]), usdtpertrade/close[i], coin, "were purchased.")
                buycount += 1
                totalcoin = usdtpertrade/close[i]
                comission += comission_rate * usdtpertrade
                transactions.append("buy")
            
            if sma50[i-1] > sma200[i-1] and sma50[i] < sma200[i] and transactions[-1] != "sell":
                print("On", calculate_time(o_time[i+1]), totalcoin, coin, "were sold.")
                sellcount += 1
                price = totalcoin * close[i]
                wallet = price
                totalcoin = 0
                comission += comission_rate * price
                transactions.append("sell")
                print("Wallet Amount after Transactions:", wallet)
                print("*********************************************************************")
                
                
    print("İnitial Wallet Amount:", 100)
    print("Total Transactions Performed:", buycount + sellcount)
    print("Total Comission:", comission)
    print("Current Wallet Amount:", wallet - comission)

In [11]:
strategyA("DOTUSDT")

*********************************************************************
On 2022-01-02 11:00:00 3.527336860670194 DOTUSDT were purchased.
On 2022-01-09 19:00:00 3.527336860670194 DOTUSDT were sold.
Wallet Amount after Transactions: 85.99647266313931
*********************************************************************
On 2022-03-17 07:00:00 5.2328623757195185 DOTUSDT were purchased.
On 2022-04-12 23:00:00 5.2328623757195185 DOTUSDT were sold.
Wallet Amount after Transactions: 90.00523286237572
*********************************************************************
On 2022-07-22 15:00:00 12.804097311139566 DOTUSDT were purchased.
On 2022-08-22 15:00:00 12.804097311139566 DOTUSDT were sold.
Wallet Amount after Transactions: 91.8053777208707
*********************************************************************
İnitial Wallet Amount: 100
Total Transactions Performed: 6
Total Comission: 4.258553124347893
Current Wallet Amount: 87.5468245965228


In [12]:
strategyA("ETHUSDT")

*********************************************************************
On 2022-02-09 11:00:00 0.03219647577376181 ETHUSDT were purchased.
On 2022-02-24 15:00:00 0.03219647577376181 ETHUSDT were sold.
Wallet Amount after Transactions: 77.0947832050304
*********************************************************************
On 2022-03-21 03:00:00 0.034948171861129945 ETHUSDT were purchased.
On 2022-04-16 11:00:00 0.034948171861129945 ETHUSDT were sold.
Wallet Amount after Transactions: 106.05512025665938
*********************************************************************
On 2022-07-14 19:00:00 0.08764625969586748 ETHUSDT were purchased.
On 2022-08-24 23:00:00 0.08764625969586748 ETHUSDT were sold.
Wallet Amount after Transactions: 147.12213506288617
*********************************************************************
İnitial Wallet Amount: 100
Total Transactions Performed: 6
Total Comission: 4.727040288934319
Current Wallet Amount: 142.39509477395185


In [13]:
# example strategy function
def strategyB(coin:str):
    usdtpertrade = 100
    buycount = 0
    sellcount = 0
    totalcoin = 0
    comission_rate = 75/10000
    comission = 0
    coin_csv = coin + ".csv"
    headers = ["Open Time", "Open", "High", "Low", "Close", "Vol", "Close Time", "QAV", "NAT", "TBBAV", "TBQAV", "Ignore"]
    coin_data = pd.read_csv(coin_csv, names=headers)
    close = coin_data["Close"]
    o_time = coin_data["Open Time"]
    macd = ta.macd(close, 12,26,9)
    rsi = ta.rsi(close)
    ema20 = ta.ema(close, length=50)
    ema50 = ta.ema(close, length=200)
    transactions = ["sell"]
    print("*********************************************************************")
    for i in range(len(close)):
        
        if pd.isna(macd["MACD_12_26_9"][i]) is False:
            
            if macd["MACD_12_26_9"][i] > 0 and transactions[-1] != "buy" and ema20[i] > ema50[i]:
                print("On", calculate_time(o_time[i+1]), usdtpertrade/close[i], coin, "were purchased.")
                buycount += 1
                totalcoin = usdtpertrade/close[i]
                comission += comission_rate * usdtpertrade
                transactions.append("buy")
                
            if macd["MACD_12_26_9"][i] < 0 and transactions[-1] != "sell" and ema20[i] < ema50[i] and rsi[i] <= 70:
                print("On", calculate_time(o_time[i+1]), totalcoin, coin, "were sold.")
                sellcount += 1
                price = close[i] * totalcoin
                wallet = price
                totalcoin = 0
                comission += comission_rate * price
                transactions.append("sell")
                print("Wallet Amount after Transactions:", wallet)
                print("*********************************************************************")
    
    print("İnitial Wallet Amount:", 100)
    print("Total Transactions Performed:", buycount + sellcount)
    print("Total Comission:", comission)
    print("Current Wallet Amount:", wallet - comission)

In [14]:
strategyB("XRPUSDT")

*********************************************************************
On 2022-02-09 03:00:00 114.11617026132603 XRPUSDT were purchased.
On 2022-02-24 11:00:00 114.11617026132603 XRPUSDT were sold.
Wallet Amount after Transactions: 73.36528586100651
*********************************************************************
On 2022-03-02 07:00:00 128.5512276642242 XRPUSDT were purchased.
On 2022-03-05 03:00:00 128.5512276642242 XRPUSDT were sold.
Wallet Amount after Transactions: 91.61845995629258
*********************************************************************
On 2022-03-12 11:00:00 125.6123602562492 XRPUSDT were purchased.
On 2022-04-09 03:00:00 125.6123602562492 XRPUSDT were sold.
Wallet Amount after Transactions: 94.82477075744252
*********************************************************************
On 2022-07-20 07:00:00 265.46323334218215 XRPUSDT were purchased.
On 2022-07-27 07:00:00 265.46323334218215 XRPUSDT were sold.
Wallet Amount after Transactions: 87.8683302362623
*********

In [15]:
strategyB("ETHUSDT")

*********************************************************************
On 2022-02-10 23:00:00 0.032040293873575404 ETHUSDT were purchased.
On 2022-02-12 15:00:00 0.032040293873575404 ETHUSDT were sold.
Wallet Amount after Transactions: 93.91298496989813
*********************************************************************
On 2022-02-16 15:00:00 0.03189141610644079 ETHUSDT were purchased.
On 2022-02-18 15:00:00 0.03189141610644079 ETHUSDT were sold.
Wallet Amount after Transactions: 91.80454388896685
*********************************************************************
On 2022-03-20 03:00:00 0.03388865543370701 ETHUSDT were purchased.
On 2022-04-16 15:00:00 0.03388865543370701 ETHUSDT were sold.
Wallet Amount after Transactions: 103.00389041764379
*********************************************************************
On 2022-07-19 07:00:00 0.06595261963805202 ETHUSDT were purchased.
On 2022-08-27 03:00:00 0.06595261963805202 ETHUSDT were sold.
Wallet Amount after Transactions: 99.46842188